#1일차 앙상블 85.4
#2일차,3일차 신경망 테스트 84

#신경망의 경우 모델의 복잡도를 낮추거나 올려도 테스트 데이터의 정확도가 84를 넘지 않음
#데이터 전처리가 필요하다고 판단



In [1]:
#필요 라이브러리들 import

import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dropout

from sklearn.model_selection import train_test_split

In [2]:
#데이터 불러오기 및 크기 확인
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

print("훈련용 데이터 크기:", train_data.shape)
print("테스트용 데이터 크기:", test_data.shape)

훈련용 데이터 크기: (17480, 16)
테스트용 데이터 크기: (15081, 15)


In [3]:
#결측치 제거 함수

def check_missing_col(dataframe):
    missing_col = []
    for col in dataframe.columns:
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            print(f'결측치가 있는 컬럼은: {col} 입니다')
            print(f'해당 컬럼에 총 {missing_values} 개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
    if missing_col == []:
        print('결측치가 존재하지 않습니다')
    return missing_col


missing_col = check_missing_col(train_data)
print(missing_col)

train_data.dropna(axis=0,subset=['workclass','occupation','native.country'],inplace = True)

결측치가 있는 컬럼은: workclass 입니다
해당 컬럼에 총 1836 개의 결측치가 존재합니다.
결측치가 있는 컬럼은: occupation 입니다
해당 컬럼에 총 1843 개의 결측치가 존재합니다.
결측치가 있는 컬럼은: native.country 입니다
해당 컬럼에 총 583 개의 결측치가 존재합니다.
[['workclass', dtype('O')], ['occupation', dtype('O')], ['native.country', dtype('O')]]


In [4]:
x_train = train_data.drop(['id', 'target'], axis=1)
y_train = train_data['target']

test_data=test_data.drop(['id'], axis=1)


print("훈련용 데이터:", x_train.shape)
print("테스트용 라벨:", y_train.shape)

훈련용 데이터: (15081, 14)
테스트용 라벨: (15081,)


In [5]:
#라벨인코딩을 하기 위함 dictionary map 생성 함수
def make_label_map(dataframe):
    label_maps = {}
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            label_map = {'unknown':0}
            for i, key in enumerate(dataframe[col].unique()):
                label_map[key] = i  #새로 등장하는 유니크 값들에 대해 1부터 1씩 증가시켜 키값을 부여해줍니다.
            label_maps[col] = label_map
    return label_maps

# 각 범주형 변수에 인코딩 값을 부여하는 함수
def label_encoder(dataframe, label_map):
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            dataframe[col] = dataframe[col].map(label_map[col])
            #dataframe[col] = dataframe[col].fillna(label_map[col]['unknown']) #혹시 모를 결측값은 unknown의 값(0)으로 채워줍니다.
    return dataframe

x_train = label_encoder(x_train, make_label_map(x_train))

#제출용 라벨링 데이터 변환
test_data = label_encoder(test_data, make_label_map(test_data))

In [6]:

X_train, X_test, Y_train,Y_test = train_test_split(x_train, y_train, random_state=22)

In [8]:
#STOME

from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=0)
X_train_over,y_train_over = smote.fit_resample(X_train,Y_train)

In [16]:
mean_vals = np.mean(X_train_over, axis=0)
std_val = np.std(X_train_over)

X_train_centered = (X_train_over  - mean_vals) / std_val

mean_vals = np.mean(X_test, axis=0)
std_val = np.std(X_test)

X_test_centered = (X_test- mean_vals) / std_val



In [19]:
#원핫 인코딩 생성
y_train_onehot = tf.keras.utils.to_categorical(y_train_over)
print(y_train_onehot[:3])

[[1. 0.]
 [1. 0.]
 [0. 1.]]


In [25]:
model = tf.keras.models.Sequential()

model.add(
    tf.keras.layers.Dense(
    units=50,
    input_dim=X_train_centered.shape[1],
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    activation='relu'
    )
)

model.add(
    tf.keras.layers.Dense(
    units=30,
    input_dim=50,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    activation='tanh'
    )
)
model.add(Dropout(0.5))


"""

model.add(
    tf.keras.layers.Dense(
    units=100,
    input_dim=150,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    activation='tanh'
    )
)

model.add(
    tf.keras.layers.Dense(
    units=50,
    input_dim=100,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    activation='tanh'
    )
)

model.add(
    tf.keras.layers.Dense(
    units=30,
    input_dim=50,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    activation='tanh'
    )
)

model.add(
    tf.keras.layers.Dense(
    units=20,
    input_dim=30,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    activation='tanh'
    )
)
"""

model.add(
    tf.keras.layers.Dense(
    units=y_train_onehot.shape[1],
    input_dim=30,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    activation='softmax'
    )
)


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                750       
_________________________________________________________________
dense_1 (Dense)              (None, 30)                1530      
_________________________________________________________________
dropout (Dropout)            (None, 30)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 62        
Total params: 2,342
Trainable params: 2,342
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.compile(optimizer='adam',metrics=['accuracy'], loss="categorical_crossentropy")

In [31]:
#early_stopping = EarlyStopping()
history = model.fit(X_train_centered, y_train_onehot, batch_size=32, epochs=10, verbose=1)

Epoch 1/10
530/530 [==============================] - 1s 2ms/step - loss: 0.3209 - accuracy: 0.8591
Epoch 2/10
530/530 [==============================] - 1s 2ms/step - loss: 0.3197 - accuracy: 0.8596
Epoch 3/10
530/530 [==============================] - 1s 2ms/step - loss: 0.3176 - accuracy: 0.8596
Epoch 4/10
530/530 [==============================] - 1s 2ms/step - loss: 0.3162 - accuracy: 0.8600
Epoch 5/10
530/530 [==============================] - 1s 2ms/step - loss: 0.3131 - accuracy: 0.8622
Epoch 6/10
530/530 [==============================] - 1s 2ms/step - loss: 0.3104 - accuracy: 0.8644
Epoch 7/10
530/530 [==============================] - 1s 2ms/step - loss: 0.3098 - accuracy: 0.8626
Epoch 8/10
530/530 [==============================] - 1s 2ms/step - loss: 0.3075 - accuracy: 0.8634
Epoch 9/10
530/530 [==============================] - 1s 2ms/step - loss: 0.3058 - accuracy: 0.8652
Epoch 10/10
530/530 [==============================] - 1s 2ms/step - loss: 0.3058 - accuracy: 0.8649

In [32]:
y_train_pred= model.predict_classes(X_train_centered, verbose=0)
correct_preds = np.sum(y_train_over == y_train_pred, axis=0)

train_acc = correct_preds / y_train_over.shape[0]

y_test_pred= model.predict_classes(X_test_centered, verbose=0)
correct_preds = np.sum(Y_test == y_test_pred, axis=0)

test_acc = correct_preds / Y_test.shape[0]

print("train 최종 결과", train_acc)
print("test 최종 결과", test_acc)

C:\Users\yong\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


train 최종 결과 0.869139010275186
test 최종 결과 0.7401219835587377


C:\Users\yong\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
